In [1]:
import pandas as pd
from pyspark.sql import SparkSession, Row


# This tutorial will use TensorFlow, TF Learn, sklearn, and pandas
# Load the packages that are needed for this tutorial
import random
import tensorflow as tf
import tensorflow.contrib.learn.python.learn as learn
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn import preprocessing
import pandas
"""

COMMENTS TO BE REMOVED AFTER WAYRS JUST FOR TESTING

"""
from tensorflow.contrib.layers.python.layers import fully_connected,dropout
import numpy as np
import random


/home/siebert/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
spark = SparkSession.builder.getOrCreate()

In [9]:
data_df = spark.sparkContext.parallelize([Row(a=i, b = i+3, c= 1+5, label = 1)\
                                          if i%2==0 else Row(a=i, b = i+3, c = 1+5, label = 0)\
                                          for i in range(1000)]).toDF()

In [10]:
def model(input1, nb_classes):
    x1 = fully_connected(320,num_outputs=320)(input1)
    x1 = fully_connected(320, num_outputs=320)(x1)
    out = fully_connected(nb_classes, num_outputs=nb_classes)(x1)
    return out


In [11]:
def to_pandas_batches(df, features, label, batch_size=10):
    num = 0
    X = []
    Y = []
    for row in df.toLocalIterator():
        if num % batch_size == 0:
            if X:
                yield pd.DataFrame.from_dict(X).as_matrix(), pd.DataFrame.from_dict(Y).as_matrix()
            X = []
            Y = []
        
        X.append(row[:-1])
        Y.append(row[-1])
        
        num += 1
    yield pd.DataFrame.from_dict(X).as_matrix(), pd.DataFrame.from_dict(Y).as_matrix()

In [12]:
from pyspark.sql.functions import rand 

features = ["a","b","c"]
label = ["label"]

split = 0.8
data_df = data_df.orderBy(rand())
train_df, test_df = data_df.randomSplit([split,1-split])
print(train_df.count())
print(test_df.count())

n_class = train_df.select("label").distinct().count()
print("Number of classes :{}".format(n_class))

814
186
Number of classes :2


In [15]:
sess = tf.Session()

#K.set_session(sess)
#K.set_learning_phase(0)

train_epoch = 10


n_input = len(features) 
print(n_input)
x = tf.placeholder(tf.float32,[None, n_input])
y = tf.placeholder(tf.float64,[None,n_class])

print(x.get_shape())
print(y.get_shape())

keep_prob = tf.placeholder(tf.float32) #dropouts
print("Setting up model !")


out =  model(x ,n_class)
print("Set up the model !")
cost = tf.reduce_mean(categorical_crossentropy(y,out))


correct_pred = tf.equal(tf.argmax(out,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
optimizer = tf.train.MomentumOptimizer(lr,0.9, use_nesterov=True).minimize(cost)
train_op = optimizer

print("Setup the initializer")
with sess.as_default():
    learning_rate = 0.001
    sess.run(tf.initialize_all_variables())
    for epoch in range(train_epoch):
        for x_train,y_train in to_pandas_batches(train_df,features,label,10):
            sess.run(train_op,feed_dict={x: x_train, y: y_train
               , keep_prob: 0.7})
            print(output)
            acc = sess.run(accuracy, feed_dict={x: x_train,y: y_train
              , keep_prob: 1.})
            #print(gaps)
            print("Accuracy :{} learning rate :{}".format(acc))

3
(?, 3)
(?, 2)
Setting up model !


ValueError: rank of shape must be at least 2 not: 0